In [12]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
import featureEng as fe
import processStocks as ps
import getStocks
import visualize
import predicts
import pandas as pd
import TS_LSTM as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [13]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18354030307206838671
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 82234572
locality {
  bus_id: 1
}
incarnation: 17234858659731132178
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


# User parameters

In [14]:
ticker = 'amd'
n_in = 5 # number of historical days to input
n_out = 5 # days into future to predict
target = 'Close'

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [15]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
dataset = getStocks.load_single(ticker)
#dataset = getStocks.join_tgt_spt(target_ticker=ticker.upper(), number_spt=20)
dataset.reset_index(level=0, inplace=True)
dataset.rename(columns=lambda x: x.replace(' ',''),inplace=True)

Loaded AMD


In [16]:
dataset.head()

,index,Date,Open,High,Low,Close,AdjClose,Volume
0,0,1980-03-17,3.125000,3.302083,3.125000,3.145833,3.145833,219600
1,1,1980-03-18,3.125000,3.125000,2.937500,3.031250,3.031250,727200
2,2,1980-03-19,3.031250,3.083333,3.020833,3.041667,3.041667,295200
3,3,1980-03-20,3.041667,3.062500,3.010417,3.010417,3.010417,159600
4,4,1980-03-21,3.010417,3.020833,2.906250,2.916667,2.916667,130800


In [17]:
ts_data=ts_lstm.ts_data.default_prep(n_in=n_in, n_out=n_out, 
                      ticker=ticker,target=target,
                      rawData=dataset)

starting Series to supverised conversion...


T:\Dropbox\Projects\LSTM-forecast\src\featureEng.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataset.drop('Date', axis=1, inplace=True)


In [18]:
ts_data.data.head()

,index(t-05),Open(t-05),High(t-05),Low(t-05),AdjClose(t-05),Volume(t-05),d1close(t-05),d2close(t-05),d1vol(t-05),d2vol(t-05),M(t-05),T(t-05),W(t-05),Th(t-05),F(t-05),Close(t-05),index(t-04),Open(t-04),High(t-04),Low(t-04),AdjClose(t-04),Volume(t-04),d1close(t-04),d2close(t-04),d1vol(t-04),d2vol(t-04),M(t-04),T(t-04),W(t-04),Th(t-04),F(t-04),Close(t-04),index(t-03),Open(t-03),High(t-03),Low(t-03),AdjClose(t-03),Volume(t-03),d1close(t-03),d2close(t-03),d1vol(t-03),d2vol(t-03),M(t-03),T(t-03),W(t-03),Th(t-03),F(t-03),Close(t-03),index(t-02),Open(t-02),High(t-02),Low(t-02),AdjClose(t-02),Volume(t-02),d1close(t-02),d2close(t-02),d1vol(t-02),d2vol(t-02),M(t-02),T(t-02),W(t-02),Th(t-02),F(t-02),Close(t-02),index(t-01),Open(t-01),High(t-01),Low(t-01),AdjClose(t-01),Volume(t-01),d1close(t-01),d2close(t-01),d1vol(t-01),d2vol(t-01),M(t-01),T(t-01),W(t-01),Th(t-01),F(t-01),Close(t-01),Close(t),Close(t+01),Close(t+02),Close(t+03),Close(t+04)
5,2.0,3.031250,3.083333,3.020833,3.041667,295200.0,0.010417,0.125000,-432000.0,-939600.0,0.0,0.0,1.0,0.0,0.0,3.041667,3.0,3.041667,3.062500,3.010417,3.010417,159600.0,-0.03125,-0.041667,-135600.0,296400.0,0.0,0.0,0.0,1.0,0.0,3.010417,4.0,3.010417,3.020833,2.906250,2.916667,130800.0,-0.093750,-0.062500,-28800.0,106800.0,0.0,0.0,0.0,0.0,1.0,2.916667,5.0,2.916667,2.916667,2.635417,2.666667,436800.0,-0.250000,-0.156250,306000.0,334800.0,1.0,0.0,0.0,0.0,0.0,2.666667,6.0,2.666667,2.750000,2.552083,2.604167,645600.0,-0.062500,0.187500,208800.0,-97200.0,0.0,1.0,0.0,0.0,0.0,2.604167,2.447917,2.375000,2.541667,2.541667,2.739583
6,3.0,3.041667,3.062500,3.010417,3.010417,159600.0,-0.031250,-0.041667,-135600.0,296400.0,0.0,0.0,0.0,1.0,0.0,3.010417,4.0,3.010417,3.020833,2.906250,2.916667,130800.0,-0.09375,-0.062500,-28800.0,106800.0,0.0,0.0,0.0,0.0,1.0,2.916667,5.0,2.916667,2.916667,2.635417,2.666667,436800.0,-0.250000,-0.156250,306000.0,334800.0,1.0,0.0,0.0,0.0,0.0,2.666667,6.0,2.666667,2.750000,2.552083,2.604167,645600.0,-0.062500,0.187500,208800.0,-97200.0,0.0,1.0,0.0,0.0,0.0,2.604167,7.0,2.604167,2.604167,2.427083,2.447917,466800.0,-0.156250,-0.093750,-178800.0,-387600.0,0.0,0.0,1.0,0.0,0.0,2.447917,2.375000,2.541667,2.541667,2.739583,2.625000
7,4.0,3.010417,3.020833,2.906250,2.916667,130800.0,-0.093750,-0.062500,-28800.0,106800.0,0.0,0.0,0.0,0.0,1.0,2.916667,5.0,2.916667,2.916667,2.635417,2.666667,436800.0,-0.25000,-0.156250,306000.0,334800.0,1.0,0.0,0.0,0.0,0.0,2.666667,6.0,2.666667,2.750000,2.552083,2.604167,645600.0,-0.062500,0.187500,208800.0,-97200.0,0.0,1.0,0.0,0.0,0.0,2.604167,7.0,2.604167,2.604167,2.427083,2.447917,466800.0,-0.156250,-0.093750,-178800.0,-387600.0,0.0,0.0,1.0,0.0,0.0,2.447917,8.0,2.375000,2.375000,2.239583,2.375000,1129200.0,-0.072917,0.083333,662400.0,841200.0,0.0,0.0,0.0,1.0,0.0,2.375000,2.541667,2.541667,2.739583,2.625000,2.635417
8,5.0,2.916667,2.916667,2.635417,2.666667,436800.0,-0.250000,-0.156250,306000.0,334800.0,1.0,0.0,0.0,0.0,0.0,2.666667,6.0,2.666667,2.750000,2.552083,2.604167,645600.0,-0.06250,0.187500,208800.0,-97200.0,0.0,1.0,0.0,0.0,0.0,2.604167,7.0,2.604167,2.604167,2.427083,2.447917,466800.0,-0.156250,-0.093750,-178800.0,-387600.0,0.0,0.0,1.0,0.0,0.0,2.447917,8.0,2.375000,2.375000,2.239583,2.375000,1129200.0,-0.072917,0.083333,662400.0,841200.0,0.0,0.0,0.0,1.0,0.0,2.375000,9.0,2.458333,2.583333,2.458333,2.541667,666000.0,0.166667,0.239584,-463200.0,-1125600.0,0.0,0.0,0.0,0.0,1.0,2.541667,2.541667,2.739583,2.625000,2.635417,2.541667
9,6.0,2.666667,2.750000,2.552083,2.604167,645600.0,-0.062500,0.187500,208800.0,-97200.0,0.0,1.0,0.0,0.0,0.0,2.604167,7.0,2.604167,2.604167,2.427083,2.447917,466800.0,-0.15625,-0.093750,-178800.0,-387600.0,0.0,0.0,1.0,0.0,0.0,2.447917,8.0,2.375000,2.375000,2.239583,2.375000,1129200.0,-0.072917,0.083333,662400.0,841200.0,0.0,0.0,0.0,1.0,0.0,2.375000,9.0,2.458333,2.583333,2.458333,2.541667,666000.0,0.166667,0.239584,-463200.0,-1125600.0,0.0,0.0,0.0,0.0,1.0,2.541667,10.0,2.541667,2.583333,2.520833,2.541667,430800.0,0.000000,-0.166667,-235200.0,22

# Build LSTM Architecture

In [21]:
ts_model = ts_lstm.build_model(ts_data, hiddenlayers=[10, 5])

Compilation Time :  0.028573036193847656
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 5, 12)             1392      
_________________________________________________________________
dropout_7 (Dropout)          (None, 5, 12)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 5, 10)             920       
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 5)                 320       
_________________________________________________________________
dropout_9 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None,

# Fit model

In [ ]:
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=100, 
                    batch_size=256, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)
visualize.plot_loss(history)

Train on 7613 samples, validate on 1904 samples
Epoch 1/100
 - 1s - loss: 87.4697 - val_loss: 76.0456
Epoch 2/100
 - 1s - loss: 87.4503 - val_loss: 76.0263
Epoch 3/100
 - 1s - loss: 87.3949 - val_loss: 76.0093
Epoch 4/100
 - 1s - loss: 87.3914 - val_loss: 75.9942
Epoch 5/100


# Visualize Results

In [ ]:
#yhat = predicts.predict_sequences_multiple(model, test_X, n_in, n_out)
#visualize.plot_results_multiple(yhat, test_y[:,0], n_out, legend=False)
# visualize.plot_single(yhat, test_y, ticker, data_set_category='test')

In [ ]:
#import predicts
## copy weights
#old_weights=model.get_weights()
#
#new_model = Sequential()
#new_model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]),
#          return_sequences=True, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(LSTM(10, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(Dense(1))
#new_model.compile(loss='mae', optimizer='adam')
## fit network
#new_model.set_weights(old_weights)
#predicts.onlineForecast(new_model, test_X, test_y)
## # # ____ # # #